In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#dxa_data
import torch
import numpy as np
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

sin = torch.sin
cos = torch.cos
exp = torch.exp
pi = torch.pi
torch.set_default_dtype(torch.float64)

epochs = 60000  # 训练代数
h = 100  # 测试点网格密度
N = 2000  # 内点配置点数
N1 = 100  # 边界点配置点数
N2 = 200  # 数据点

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)

setup_seed(123456)

def gen_testdata():
    # 修改为加载2D数据，格式应为 (x, y, t, exact)
    # 这里假设数据已处理成适当格式

    data = np.load("/content/drive/MyDrive/dxa_data_3_1.7.npz")
    #data = np.load("/content/drive/MyDrive/dxa_data_3_1.8.npz")
    #data = np.load("/content/drive/MyDrive/dxa_data_3_1.9.npz")
    #data = np.load("/content/drive/MyDrive/dxa_data_3_2.npz")
    #data = np.load("/content/drive/MyDrive/dxa_data_3_2.1.npz")
    #data = np.load("/content/drive/MyDrive/dxa_data_3_2.2.npz")

    t, x, y, usol = data["t"], data["x"], data["y"], data["u_sol"]  # 需要确保数据格式正确
    xx, yy, tt = np.meshgrid(x, y, t)
    X = np.vstack((np.ravel(xx), np.ravel(yy), np.ravel(tt))).T
    z = usol.flatten()[:, None]
    return X, z

def l2_relative_error(z_true, z_pred):
    return np.linalg.norm(z_true - z_pred) / np.linalg.norm(z_true)

# Domain and Sampling (修改为3D采样)
def interior(n=N):
    # 内点采样 x∈[-3,3], y∈[-3,3], t∈[0,1]
    x = torch.rand(n, 1)*6 - 3  # [-3,3]
    y = torch.rand(n, 1)*6 - 3  # [-3,3]
    t = torch.rand(n, 1)        # [0,1]
    cond = torch.zeros_like(t)
    return x.requires_grad_(True), y.requires_grad_(True), t.requires_grad_(True), cond

def ini(n=N1):
    # 初始条件 t=0
    x = torch.rand(n, 1)*6 - 3
    y = torch.rand(n, 1)*6 - 3
    t = torch.zeros_like(x)
    cond = torch.where(x**2+y**2<0.25, torch.full_like(x,0.5), torch.zeros_like(x))  # 示例条件
    return x.requires_grad_(True), y.requires_grad_(True), t.requires_grad_(True), cond

def left(n=N1):
    # 边界 u(-3,y,t)=0
    t = torch.rand(n, 1)
    x = -3 * torch.ones_like(t)
    y = torch.rand(n, 1)*6 - 3
    cond = torch.zeros_like(t)
    return x.requires_grad_(True),y.requires_grad_(True), t.requires_grad_(True), cond

def right(n=N1):
    # 边界 u(3,y,t)=0
    t = torch.rand(n, 1)
    x = 3 * torch.ones_like(t)
    y = torch.rand(n, 1)*6 - 3
    cond = torch.zeros_like(t)
    return x.requires_grad_(True),y.requires_grad_(True), t.requires_grad_(True), cond

def up(n=N1):
    # 边界 u(x,3,t)=0
    t = torch.rand(n, 1)
    y = 3 * torch.ones_like(t)
    x = torch.rand(n, 1)*6 - 3
    cond = torch.zeros_like(t)
    return x.requires_grad_(True), y.requires_grad_(True),t.requires_grad_(True), cond

def down(n=N1):
    # 边界 u(x,-3,t)=0
    t = torch.rand(n, 1)
    y = -3 * torch.ones_like(t)
    x = torch.rand(n, 1)*6 - 3
    cond = torch.zeros_like(t)
    return x.requires_grad_(True),y.requires_grad_(True), t.requires_grad_(True), cond

def data_interior(n=N2):
    # 内点数据
    X, Y = gen_testdata()
    ids = np.random.randint(X.shape[0], size=n)
    xy_t = X[ids]
    u_real = Y[ids]
    xy_t = torch.from_numpy(xy_t)
    cond = torch.from_numpy(u_real)
    xy_t.requires_grad_(True)
    return xy_t, cond

# Neural Network (输入维度改为3)
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(3, 64),  # 输入x,y,t
            torch.nn.Tanh(),
            torch.nn.Linear(64, 64),
            torch.nn.Tanh(),
            torch.nn.Linear(64, 64),
            torch.nn.Tanh(),
            torch.nn.Linear(64, 1),
        )


    def forward(self, x):
        features = self.net(x)
        u_val = torch.abs(features)
        return u_val


# Loss functions
loss = torch.nn.MSELoss()


def gradients(u, x, order=1):
    if order == 1:
        return torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),
                                 create_graph=True,
                                 only_inputs=True)[0]
    else:
        return gradients(gradients(u, x), x, order=order-1)

v = torch.nn.Parameter(torch.randn(1), requires_grad=True)
#v = torch.nn.Parameter(torch.tensor([1.9]), requires_grad=True) ##33
print('Initial v: ', v)

def l_interior(u):
    x, y, t, cond = interior()
    inputs = torch.cat([x, y, t], dim=1)
    u_val = u(inputs)  # 原始网络输出

    # 计算缩放后的u'及其导数
    u_prime = u_val * 1
    u_t = gradients(u_prime, t, 1)
    u_x = gradients(u_prime, x, 1)
    u_y = gradients(u_prime, y, 1)
    u_xx = gradients(u_prime, x, 2)
    u_yy = gradients(u_prime, y, 2)


    residual = u_t - 6*u_prime*(u_x**2+u_y**2)-3*(u_prime**2)*(u_xx + u_yy) - v*u_prime
    return loss(residual, cond)

def l_ini(u):
    x, y, t, cond = ini()
    u_val = u(torch.cat([x, y, t], dim=1))
    return loss(u_val, cond)

def l_left(u):
    # 边界条件损失
    x, y, t, cond = left()
    u_val = u(torch.cat([x, y, t], dim=1))
    return loss(u_val, cond)

def l_right(u):
    # 边界条件损失
    x, y, t, cond = right()
    u_val = u(torch.cat([x, y, t], dim=1))
    return loss(u_val, cond)

def l_up(u):
    # 边界条件损失
    x, y, t, cond = up()
    u_val = u(torch.cat([x, y, t], dim=1))
    return loss(u_val, cond)

def l_down(u):
    # 边界条件损失
    x, y, t, cond = down()
    u_val = u(torch.cat([x, y, t], dim=1))
    return loss(u_val, cond)

def l_data(u):
    xyt, cond = data_interior()
    u_val = u(xyt)
    return loss(u_val, cond)



# Training
#u = MLP()
#opt = torch.optim.Adam(u.parameters(), lr=1e-3)
u = MLP()
opt = torch.optim.RAdam(list(u.parameters()) + [v], lr=1e-3)

scheduler = StepLR(opt, step_size=1000, gamma=0.9)  # 对所有参数应用调度




# 记录训练过程
L2_error = []
v_values = []
iterations = []

# 获取测试数据
xy_t, u_real = gen_testdata()
xy_t = torch.from_numpy(xy_t)

for i in range(epochs):
    opt.zero_grad()
    #opt1.zero_grad()
    l_i = l_interior(u)
    l_d = l_data(u)
    l_initial = l_ini(u)
    l_u = l_up(u)
    l_dn = l_down(u)
    l_l = l_left(u)
    l_r = l_right(u)


    # 计算各项损失
    l = 10*l_i + l_initial + l_dn + l_u + l_l + l_r + 40*l_data(u)
    l.backward()

    torch.nn.utils.clip_grad_norm_(u.parameters(), 0.1)

    opt.step()
    #opt1.step()
    scheduler.step()
    v.data.clamp_(min=1e-6)

    # 记录参数
    v_values.append(v.item())
    iterations.append(i+1)

    if (i+1) % 1000 == 0 or i == 0:
        u_pred = u(xy_t)
        error = l2_relative_error(u_real, u_pred.detach().numpy())
        L2_error.append(error)
        print(f"Epoch {i+1}, L2相对误差: {error}, v: {v.item()}")
        print(f"  Interior loss: {l_i.item():.6f}")
        print(f"  Data loss: {l_d.item():.6f}")
        print(f"  Initial loss: {l_initial.item():.6f}")
        print(f"  Up boundary loss: {l_u.item():.6f}")
        print(f"  Down boundary loss: {l_dn.item():.6f}")
        print(f"  Left boundary loss: {l_l.item():.6f}")
        print(f"  Right boundary loss: {l_r.item():.6f}")
        print(f"  loss: {l.item():.6f}")


print('Final v: ', v)
plt.figure(figsize=(10, 6))
plt.plot(iterations, v_values, 'b-', linewidth=2)
plt.xlabel('number of iterations', fontsize=14)
plt.ylabel('v', fontsize=14)
plt.title('changes of v W.R.T iterations', fontsize=16)
plt.grid(True)
plt.show()

Initial v:  Parameter containing:
tensor([1.8645], requires_grad=True)
Epoch 1, L2相对误差: 1.3757716663816213, v: 1.8642095136303796
  Interior loss: 0.031442
  Data loss: 0.007728
  Initial loss: 0.011948
  Up boundary loss: 0.010251
  Down boundary loss: 0.002615
  Left boundary loss: 0.016440
  Right boundary loss: 0.006123
  loss: 0.632939
Epoch 1000, L2相对误差: 0.4075550584772359, v: 1.577906505004686
  Interior loss: 0.001458
  Data loss: 0.000595
  Initial loss: 0.003971
  Up boundary loss: 0.000057
  Down boundary loss: 0.000113
  Left boundary loss: 0.000187
  Right boundary loss: 0.000082
  loss: 0.052486
Epoch 2000, L2相对误差: 0.3172550286395513, v: 1.36233018982723
  Interior loss: 0.000997
  Data loss: 0.000512
  Initial loss: 0.001589
  Up boundary loss: 0.000018
  Down boundary loss: 0.000136
  Left boundary loss: 0.000077
  Right boundary loss: 0.000061
  loss: 0.036588
Epoch 3000, L2相对误差: 0.29755360311232787, v: 1.3221613970985047
  Interior loss: 0.001168
  Data loss: 0.000308

KeyboardInterrupt: 